In [187]:
import datetime
import numpy as np
import pandas as pd
import warnings

import pdb

pd.options.display.max_columns = 200
pd.options.display.max_rows = 200
warnings.filterwarnings('ignore')

wbpath = 'data/DP Beh Data 2022-23.xlsm'
#read in necessary sheets 
dfcolnmes = pd.read_excel(wbpath, sheet_name='Oct', skiprows=0)
df = pd.read_excel(wbpath, sheet_name='Oct', skiprows=1)

dfinfo = pd.read_excel(wbpath, sheet_name=0)
dfstudet = pd.read_excel(wbpath, sheet_name=2, header=None)

#get years variable 
year = dfinfo.iloc[3,1]
year1, year2 = str.split(year, '-')

df.head()

,Date,Shift,No Data,Unnamed: 3,Subcategory,Unnamed: 5,Unnamed: 6,Freq,Duration Specific,Time Sampling %,Duration Intervals,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Intensity,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Comments,Subcategory.1,Unnamed: 23,Unnamed: 24,Freq.1,Duration Specific.1,Time Sampling %.1,Duration Intervals.1,Unnamed: 29,Unnamed: 30,Unnamed: 31,Unnamed: 32,Unnamed: 33,Intensity.1,Unnamed: 35,Unnamed: 36,Unnamed: 37,Unnamed: 38,Comments.1,Subcategory.2,Unnamed: 41,Unnamed: 42,Freq.2,Duration Specific.2,Time Sampling %.2,Duration Intervals.2,Unnamed: 47,Unnamed: 48,Unnamed: 49,Unnamed: 50,Unnamed: 51,Intensity.2,Unnamed: 53,Unnamed: 54,Unnamed: 55,Unnamed: 56,Comments.2,Subcategory.3,Unnamed: 59,Unnamed: 60,Freq.3,Duration Specific.3,Time Sampling %.3,Duration Intervals.3,Unnamed: 65,Unnamed: 66,Unnamed: 67,Unnamed: 68,Unnamed: 69,Intensity.3,Unnamed: 71,Unnamed: 72,Unnamed: 73,Unnamed: 74,Comments.3,Subcategory.4,Unnamed: 77,Unnamed: 78,Freq.4,Duration Specific.4,Time Sampling %.4,Duration Intervals.4,Unnamed: 83,Unnamed: 84,Unnamed: 85,Unnamed: 86,Unnamed: 87,Intensity.4,Unnamed: 89,Unnamed: 90,Unnamed: 91,Unnamed: 92,Comments.4,Unnamed: 94,Unnamed: 95,Unnamed: 96,Unnamed: 97,Unnamed: 98,Unnamed: 99,Unnamed: 100,Unnamed: 101,Unnamed: 102,Unnamed: 103,Unnamed: 104,Unnamed: 105,Unnamed: 106,Unnamed: 107,Unnamed: 108,Unnamed: 109,Unnamed: 110,Unnamed: 111,Unnamed: 112,Unnamed: 113,Unnamed: 114,Unnamed: 115,Unnamed: 116,Unnamed: 117
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Not Specified,<5,6-10,11-15,16-20,20+,Not Specified,1.0,2.0,3.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Not Specified,0.0,0.0,0.0,0.0,0.0,Not Specified,1.0,2.0,3.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Not Specified,<5,6-10,11-15,16-20,20+,Not Specified,1.0,2.0,3.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Not Specified,<5,6-10,11-15,16-20,20+,Not Specified,1.0,2.0,3.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Not Specified,0.0,0.0,0.0,0.0,0.0,Not Specified,1.0,2.0,3.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,7-3,LOA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,3-11,LOA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,11-7,LOA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2,7-3,LOA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [188]:
dfcolnmes.head()

,October,Unnamed: 1,Unnamed: 2,Unnamed: 3,Aggression,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Self Mutilation,Unnamed: 23,Unnamed: 24,Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28,Unnamed: 29,Unnamed: 30,Unnamed: 31,Unnamed: 32,Unnamed: 33,Unnamed: 34,Unnamed: 35,Unnamed: 36,Unnamed: 37,Unnamed: 38,Unnamed: 39,SIB,Unnamed: 41,Unnamed: 42,Unnamed: 43,Unnamed: 44,Unnamed: 45,Unnamed: 46,Unnamed: 47,Unnamed: 48,Unnamed: 49,Unnamed: 50,Unnamed: 51,Unnamed: 52,Unnamed: 53,Unnamed: 54,Unnamed: 55,Unnamed: 56,Unnamed: 57,Disruptive,Unnamed: 59,Unnamed: 60,Unnamed: 61,Unnamed: 62,Unnamed: 63,Unnamed: 64,Unnamed: 65,Unnamed: 66,Unnamed: 67,Unnamed: 68,Unnamed: 69,Unnamed: 70,Unnamed: 71,Unnamed: 72,Unnamed: 73,Unnamed: 74,Unnamed: 75,Insert,Unnamed: 77,Unnamed: 78,Unnamed: 79,Unnamed: 80,Unnamed: 81,Unnamed: 82,Unnamed: 83,Unnamed: 84,Unnamed: 85,Unnamed: 86,Unnamed: 87,Unnamed: 88,Unnamed: 89,Unnamed: 90,Unnamed: 91,Unnamed: 92,Unnamed: 93,Unnamed: 94,Bite Rel,F-Arm Catch,F-Choke Windmill,B-Choke,F-Hair Stab/Rel,B-Hair Stab/Rel,Deflection,Bite prevention,Front hair Pull Stab,Touch,Block Punch,1 Escort,2 Escort,2 Removal,1 E Seat Var,2 E Seat Var,Stand Wrap,One Arm Release,Mitts wear,Splints wear,Stuck/Combo,Frozen,Pacing
0,Date,Shift,No Data,NaN,Subcategory,NaN,NaN,Freq,Duration Specific,Time Sampling %,Duration Intervals,NaN,NaN,NaN,NaN,NaN,Intensity,NaN,NaN,NaN,NaN,Comments,Subcategory,NaN,NaN,Freq,Duration Specific,Time Sampling %,Duration Intervals,NaN,NaN,NaN,NaN,NaN,Intensity,NaN,NaN,NaN,NaN,Comments,Subcategory,NaN,NaN,Freq,Duration Specific,Time Sampling %,Duration Intervals,NaN,NaN,NaN,NaN,NaN,Intensity,NaN,NaN,NaN,NaN,Comments,Subcategory,NaN,NaN,Freq,Duration Specific,Time Sampling %,Duration Intervals,NaN,NaN,NaN,NaN,NaN,Intensity,NaN,NaN,NaN,NaN,Comments,Subcategory,NaN,NaN,Freq,Duration Specific,Time Sampling %,Duration Intervals,NaN,NaN,NaN,NaN,NaN,Intensity,NaN,NaN,NaN,NaN,Comments,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Not Specified,<5,6-10,11-15,16-20,20+,Not Specified,1.0,2.0,3.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Not Specified,0.0,0.0,0.0,0.0,0.0,Not Specified,1.0,2.0,3.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Not Specified,<5,6-10,11-15,16-20,20+,Not Specified,1.0,2.0,3.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Not Specified,<5,6-10,11-15,16-20,20+,Not Specified,1.0,2.0,3.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Not Specified,0.0,0.0,0.0,0.0,0.0,Not Specified,1.0,2.0,3.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1,7-3,LOA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,3-11,LOA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,11-7,LOA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N

In [189]:
idx1 = [8,26,44,62,80]
idx2 = [21,39,57,75,93]
tgt = [4,22,40,58,76]
dfdurinlist=[]

df.columns = df.columns.str.replace('.1','')
df.columns = df.columns.str.replace('.2','') 
df.columns = df.columns.str.replace('.3','') 
df.columns = df.columns.str.replace('.4','') 
df.columns = df.columns.str.replace('.5','')

#create 'month number' variable 
month_name = dfcolnmes.columns[0]
datetime_object = datetime.datetime.strptime(month_name, "%B")
month_num = datetime_object.month

#if month number is between 7 and 12 year1 is selected, else year2 
if 7 <= month_num <= 12:
    year_num = year1
else:
    year_num = year2

#create date string 
date = str(year_num) + '-' + str(month_num)
print(date)

for i, enum in enumerate(idx1):
    
    dfiter = df.iloc[:,[0,1,2] + list(range(enum,idx2[i]))]
    dfiter.insert(3,'Target', str(dfcolnmes.columns[tgt[i]]))
    dfiter.drop(0, inplace=True)

    #print(pd.DataFrame(dfiter.head()))
    dfiter.columns.values[7] = '<5'
    dfiter.columns.values[8] = '6-10'
    dfiter.columns.values[9] = '11-15'
    dfiter.columns.values[10] = '16-20'
    dfiter.columns.values[11] = '20+'
    dfiter.columns.values[12] = 'Not Specified'
    dfiter.columns.values[13] = '1'
    dfiter.columns.values[14] = '2'
    dfiter.columns.values[15] = '3'
    dfiter.columns.values[16] = '4'

    dfiter = dfiter.drop(dfiter.index[np.where(dfiter.index >= 94)])
    dfiter.iloc[:,0] = dfiter.iloc[:,0].fillna(method='ffill')

    #cut off dataframe rows based on number of days in month
    if month_num == 4 or month_num == 6 or month_num == 9 or month_num == 11:
        dfiter = dfiter.drop(dfiter.index[np.where(dfiter.index >= 90)])
    elif month_num == 2:
        dfiter = dfiter.drop(dfiter.index[np.where(dfiter.index >= 84)])
    else:
        dfiter = dfiter
    
    dfdurinlist.append(dfiter)

#NOTE; need to melt dataframe and clean up !!!!!

print(dfdurin.shape)
dfdurin = pd.concat(dfdurinlist)
pd.DataFrame(dfdurinlist[0].head())
dfdurin.head(200)

2022-10
(465, 17)


,Date,Shift,No Data,Target,Duration Specific,Time Sampling %,Duration Intervals,<5,6-10,11-15,16-20,20+,Not Specified,1,2,3,4
1,1,7-3,LOA,Aggression,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1,3-11,LOA,Aggression,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1,11-7,LOA,Aggression,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2,7-3,LOA,Aggression,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,2,3-11,LOA,Aggression,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,2,11-7,LOA,Aggression,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,3,7-3,LOA,Aggression,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,3,3-11,No Behaviors,Aggression,0.0,0.0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0
9,3,11-7,No Data Recorded,Aggression,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10,4,7-3,NaN,Aggression,NaN,NaN,0,2,0,0,0,0,1,1.0,NaN,NaN,NaN


In [190]:
# dfdurin1 = df.iloc[:,[0,1,2] + list(range(8,21))]
# dfdurin1.insert(3,'Target', str(dfcolnmes.columns[4]))
# dfdurin2 = df.iloc[:,[0,1,2] + list(range(26,39))]
# dfdurin2.insert(3, 'Target', str(dfcolnmes.columns[22]))
# dfdurin3 = df.iloc[:,[0,1,2] + list(range(44,57))]
# dfdurin3.insert(3, 'Target', str(dfcolnmes.columns[40]))
# dfdurin4 = df.iloc[:,[0,1,2] + list(range(62,75))]
# dfdurin4.insert(3, 'Target', str(dfcolnmes.columns[58]))
# dfdurin5 = df.iloc[:,[0,1,2] + list(range(80,93))]
# dfdurin5.insert(3, 'Target', str(dfcolnmes.columns[76]))

# dfdurin = pd.concat([dfdurin1,dfdurin2,dfdurin3,dfdurin4,dfdurin5])              
# print(dfdurin.shape)
# dfdurin2.head()

In [191]:
#NOTE; rename 'Duration Intervals' columns with the behavior name, same as - `'Unnamed: 7':str(dfbeh.columns[3])+'_beh'`

# dfdurin = dfdurin.rename(columns = {'Unnamed: 1':'Shift','Unnamed: 2':'No Data',
#                                 'Unnamed: 11':str(dfdurin.iloc[0,7]),'Unnamed: 46':str(dfcolnmes.columns[7]),
#                                 'Unnamed: 64':str(dfcolnmes.columns[9]),'Unnamed: 79':str(dfcolnmes.columns[11])})

In [192]:
df = pd.read_excel(wbpath, sheet_name='July', skiprows=[1,2])
dfinfo = pd.read_excel(wbpath, sheet_name=0)
dfstudet = pd.read_excel(wbpath, sheet_name=2, header=None)

#get years variable 
year = dfinfo.iloc[3,1]
year1, year2 = str.split(year, '-')

#get name
# name = dfstudet.iloc[0,0]
# print(f"name is {name}")

# pdb.set_trace()
#select necessary columns and front fill NaNs in month column 
dfbeh = df.iloc[:, [0,1,2,4,7,22,25,40,43,58,61,76,79]]
#print(dfbeh.head())
dfbeh = dfbeh.rename(columns = {'Unnamed: 1':'Shift','Unnamed: 2':'No Data','Unnamed: 7':str(dfbeh.columns[3])+'_beh','Unnamed: 25':str(dfbeh.columns[5])+'_beh',
                                'Unnamed: 43':str(dfbeh.columns[7])+'_beh','Unnamed: 61':str(dfbeh.columns[9])+'_beh','Unnamed: 79':str(dfbeh.columns[11])+'_beh'})

dfbeh.head()

,July,Shift,No Data,Aggression,Aggression_beh,Self Mutilation,Self Mutilation_beh,SIB,SIB_beh,Disruptive,Disruptive_beh,Insert,Insert_beh
0,1,7-3,NaN,"hitting, grabbing",2.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,NaN
1,NaN,3-11,No Behaviors,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0
2,NaN,11-7,No Behaviors,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0
3,2,7-3,No Behaviors,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0
4,NaN,3-11,No Behaviors,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0
